# Activity 9

This activity tests your skills
in approaching an engineering problem
as solving a system of nonlinear algebraic equations.

## Scenario

Referring to [the diagram](./act-09.png),
a device has an active power consumption $P$
when operating at a lagging power factor $\Phi$.
The requisite power is conveyed by the
resistance $R$,
inductance $L$
and the capacitor $C$,
from a source
configured to provide a steady RMS voltage $U$
at a steady frequency $F$.

You are to solve for the following quantities.
- $V$, the RMS operating voltage of the device
- $\delta_{\text{l}}$,
  the phase angle of the voltage phasor at which the device receives power
- $\delta_{\text{s}}$
  the phase angle of the voltage phasor provided by the source

## Modelling

Cast the original task as
solving a system of nonlinear algebraic equations
in three unknowns
(namely, $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$).
Introduce a vector $\boldsymbol{x}$ 
to collect the unknowns in the order presented above,
so that the system of equations can be written in vector form as
$\boldsymbol{f}\!\left(\boldsymbol{x}\right) = \boldsymbol{0}$.
Set the voltage phasor across the input terminals of the load
to have the reference phase angle.

## Core code

In [1]:
import math as mt
import numpy as np
import scipy as sp

from math import sin, cos, tan, pi, acos

def sec(x):
    return 1/cos(x)

Define a Python function
`f()` that takes

- one positional argument,
  `x`, representing $\boldsymbol{x}$;
  and
- four keyword arguments,
  `U`, `F`, `R`, `L`, `C`, `P`, and `Phi`
  representing $U$, $F$, $R$, $L$, $C$, $P$, and $\Phi$ respectively.

`f()` returns a NumPy array representing
$\boldsymbol{f}\!\left(\boldsymbol{x}\right)$.

Moreover, define a Python function `dfdx()`
that takes the same arguments as `f()`,
and returns a NumPy array representing
the Jacobian of $\boldsymbol{f}\!\left(\boldsymbol{x}\right)$ w.r.t. $\boldsymbol{x}$.


In [2]:
def f(x, U, F, R, L, C, P, Phi):
    Pw = P*10**-3
    
    f1 = R*(Pw/x[0]) - (((2*pi*F*L)*(Pw*tan(acos(Phi)) - (x[0]**2)*2*pi*F*C ))/(x[0])) - U*cos(x[1]) + x[0]
    
    f2 = (R/x[0])*(Pw*tan(acos(Phi)) - (x[0]**2)*2*pi*F*C) + ((2*pi*F*L*Pw)/(x[0])) - U*sin(x[1])
    
    return np.array([f1, f2])

def dfdx(x, U, F, R, L, C, P, Phi):
    Pw = P*10**-3

    df1dV = ((Pw*R - ((2*pi*F*L)*((-2*pi*F*L*(x[0]**2)) + Pw*tan(acos(Phi)))))/(x[0]**2)) + 8*((pi*F*L)**2)*x[0] + 1

    df1dt = U*sin(x[1])

    df2dV = -((R*((-2*pi*F*C)*(x[0]**2) + Pw*tan(acos(Phi))))/(x[0]**2)) - 4*pi*F*C*R - (2*pi*F*L*Pw)/(x[0]**2)

    df2dt = -U*cos(x[1])
    
    return np.array([
            [df1dV, df1dt],
            [df2dV, df2dt]
    ])


##(RP - 2piFL(Ptan(arccos(phi)) - V^2 2piFL))/V - Ucos(t_S) + V = f1

##(R/V)(Ptan(arccos(phi)) - V^2 2piFC) + (2piFLP)/V - Usin(t_S) = f2

## Assessment

Every time you solve for $\boldsymbol{x}$,
- define Python variables
  `U`, `F`, `R`, `L`, `C`, `P`, and `PHI`
  for storing the values of parameters
  $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$,
  respectively;
- define a Python variable `x_ini`
  for storing your initial guesses of the variables;
- use [`scipy.optimize.root()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html),
  particularly its `args` keyword argument;
- define a Python variable `x_out` to store $\boldsymbol{x}$;
- verify if $\boldsymbol{f}\!\left(\boldsymbol{x}\right) = \boldsymbol{0}$ is sufficiently satisfied;
- print the value of $V$ in kilovolts;
  and
- print the values of $\delta_{\text{l}}$ and $\delta_{\text{s}}$ in degrees.

### Task 1

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
13.8 kilovolts,
60 hertz,
0.21 ohms, 5.570 millihenrys, 1.560 millifarads,
40 kilowatts, 0.87.

In [3]:
U = 1
values1 = (13.8, 60, 0.21, 5.57*10**-3, 1.56*10**-3, 40, 0.87)
x_ini1 = np.array([U, 0])
x_out1 = sp.optimize.root(f, x_ini1, args=values1, jac=dfdx)
print(x_out1)
print("")
print(f"f(x) is {f(x_out1.x, values1[0], values1[1], values1[2], values1[3], values1[4], values1[5], values1[6])}")
print("")
print(f"The Load Voltage is: {x_out1.x[0]} kV")
print(f"The Source Phase Angle is: {np.degrees(x_out1.x[1])} degrees")
print(f"The Load Phase Angle is: 0 degrees")

 message: The solution converged.
 success: True
  status: 1
     fun: [ 8.882e-16 -2.220e-16]
       x: [ 6.168e+00 -5.419e-02]
  method: hybr
    nfev: 12
    njev: 1
    fjac: [[-9.984e-01  5.609e-02]
           [-5.609e-02 -9.984e-01]]
       r: [-2.245e+00 -9.672e-01  1.376e+01]
     qtf: [ 6.303e-13  2.351e-14]

f(x) is [ 8.88178420e-16 -2.22044605e-16]

The Load Voltage is: 6.168472647362725 kV
The Source Phase Angle is: -3.1047589317533375 degrees
The Load Phase Angle is: 0 degrees


### Task 2

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
13.8 kilovolts,
50 hertz,
0.17 ohms, 6.685 milllihenrys, 1.872 millifarads,
32 kilowatts, 0.97.

In [4]:
U = 1
values2 = (13.8, 50, 0.17, 6.685*10**-3, 1.872*10**-3, 32, 0.97)
x_ini2 = np.array([U, 0.1])
x_out2 = sp.optimize.root(f, x_ini2, args=values2, jac=dfdx)
print(x_out2)
print("")
print(f"f(x) is {f(x_out2.x, values2[0], values2[1], values2[2], values2[3], values2[4], values2[5], values2[6])}")
print("")
print(f"The Load Voltage is: {x_out2.x[0]} kV")
print(f"The Source Phase Angle is: {np.degrees(x_out2.x[1])} degrees")
print(f"The Load Phase Angle is: 0 degrees")

 message: The solution converged.
 success: True
  status: 1
     fun: [-2.665e-15 -1.110e-16]
       x: [ 6.169e+00 -4.390e-02]
  method: hybr
    nfev: 12
    njev: 1
    fjac: [[-9.990e-01  4.459e-02]
           [-4.459e-02 -9.990e-01]]
       r: [-2.271e+00 -3.298e+00  1.360e+01]
     qtf: [ 2.340e-11  1.423e-12]

f(x) is [-2.66453526e-15 -1.11022302e-16]

The Load Voltage is: 6.169059947462805 kV
The Source Phase Angle is: -2.5154104160324464 degrees
The Load Phase Angle is: 0 degrees


### Task 3

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
32.5 kilovolts,
60 hertz,
0.21 ohms, 5.570 millihenrys, 1.560 millifarads,
214 kilowatts, 0.87.

In [5]:
U = 1
values3 = (32.5, 60, 0.21, 5.57*10**-3, 1.56*10**-3, 214, 0.87)
x_ini3 = np.array([U, 0])
x_out3 = sp.optimize.root(f, x_ini3, args=values3, jac=dfdx)
print(x_out3)
print("")
print(f"f(x) is {f(x_out3.x, values3[0], values3[1], values3[2], values3[3], values3[4], values3[5], values3[6])}")
print("")
print(f"The Load Voltage is: {x_out3.x[0]} kV")
print(f"The Source Phase Angle is: {np.degrees(x_out3.x[1])} degrees")
print(f"The Load Phase Angle is: 0 degrees")

 message: The solution converged.
 success: True
  status: 1
     fun: [ 1.776e-15 -2.220e-16]
       x: [ 1.453e+01 -5.422e-02]
  method: hybr
    nfev: 12
    njev: 1
    fjac: [[-9.985e-01  5.564e-02]
           [-5.564e-02 -9.985e-01]]
       r: [-2.255e+00 -3.072e+00  3.228e+01]
     qtf: [-2.958e-11 -2.193e-12]

f(x) is [ 1.77635684e-15 -2.22044605e-16]

The Load Voltage is: 14.52693478182052 kV
The Source Phase Angle is: -3.1068181542872213 degrees
The Load Phase Angle is: 0 degrees


### Task 4

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
32.5 kilovolts,
50 hertz,
0.17 ohms, 6.685 milllihenrys, 1.872 millifarads,
173 kilowatts, 0.97.

In [6]:

values4 = (32.5, 50, 0.17, 6.685*10**-3, 1.872*10**-3, 173, 0.97)
x_ini4 = np.array([1, 0])
x_out4 = sp.optimize.root(f, x_ini4, args=values4, jac=dfdx)
print(x_out4)
print("")
print(f"f(x) is {f(x_out4.x, values4[0], values4[1], values4[2], values4[3], values4[4], values4[5], values4[6])}")
print("")
print(f"The Load Voltage is: {x_out4.x[0]} kV")
print(f"The Source Phase Angle is: {np.degrees(x_out4.x[1])} degrees")
print(f"The Load Phase Angle is: 0 degrees")

 message: The solution converged.
 success: True
  status: 1
     fun: [ 9.965e-13  1.155e-14]
       x: [ 1.453e+01 -4.392e-02]
  method: hybr
    nfev: 11
    njev: 1
    fjac: [[-9.990e-01  4.523e-02]
           [-4.523e-02 -9.990e-01]]
       r: [-2.247e+00 -2.382e+00  3.237e+01]
     qtf: [ 1.040e-07  5.887e-09]

f(x) is [9.96536187e-13 1.15463195e-14]

The Load Voltage is: 14.528521138637798 kV
The Source Phase Angle is: -2.5165662091826935 degrees
The Load Phase Angle is: 0 degrees


### Task 5

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
69.0 kilovolts,
60 hertz,
0.21 ohms, 5.570 millihenrys, 1.560 millifarads,
421 kilowatts, 0.87.

In [7]:
U = 1
values5 = (69, 60, 0.21, 5.57*10**-3, 1.56*10**-3, 421, 0.87)
x_ini5 = np.array([1, 0])
x_out5 = sp.optimize.root(f, x_ini5, args=values5, jac=dfdx)
print(x_out5)
print("")
print(f"f(x) is {f(x_out5.x, values5[0], values5[1], values5[2], values5[3], values5[4], values5[5], values5[6])}")
print("")
print(f"The Load Voltage is: {x_out5.x[0]} kV")
print(f"The Source Phase Angle is: {np.degrees(x_out5.x[1])} degrees")
print(f"The Load Phase Angle is: 0 degrees")

 message: The solution converged.
 success: True
  status: 1
     fun: [ 1.561e-11 -3.522e-13]
       x: [ 3.083e+01 -5.478e-02]
  method: hybr
    nfev: 11
    njev: 1
    fjac: [[-9.985e-01  5.550e-02]
           [-5.550e-02 -9.985e-01]]
       r: [-2.246e+00 -6.750e+00  6.878e+01]
     qtf: [ 2.128e-07  7.166e-09]

f(x) is [ 1.56141766e-11 -3.52162743e-13]

The Load Voltage is: 30.83314453262459 kV
The Source Phase Angle is: -3.1384445896001534 degrees
The Load Phase Angle is: 0 degrees


### Task 6

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
32.5 kilovolts,
50 hertz,
0.17 ohms, 6.685 milllihenrys, 1.872 millifarads,
317 kilowatts, 0.97.

In [8]:
U = 1
values6 = (32.5, 50, 0.17, 6.685*10**-3, 1.872*10**-3, 317, 0.97)
x_ini6 = np.array([1, 0])
x_out6 = sp.optimize.root(f, x_ini6, args=values6, jac=dfdx)
print(x_out6)
print("")
print(f"f(x) is {f(x_out6.x, values6[0], values6[1], values6[2], values6[3], values6[4], values6[5], values6[6])}")
print("")
print(f"The Load Voltage is: {x_out6.x[0]} kV")
print(f"The Source Phase Angle is: {np.degrees(x_out6.x[1])} degrees")
print(f"The Load Phase Angle is: 0 degrees")

 message: The solution converged.
 success: True
  status: 1
     fun: [ 5.329e-15 -2.220e-16]
       x: [ 1.453e+01 -4.327e-02]
  method: hybr
    nfev: 12
    njev: 1
    fjac: [[-9.990e-01  4.574e-02]
           [-4.574e-02 -9.990e-01]]
       r: [-2.250e+00 -2.699e+00  3.232e+01]
     qtf: [-2.044e-12 -1.392e-13]

f(x) is [ 5.32907052e-15 -2.22044605e-16]

The Load Voltage is: 14.530511403949829 kV
The Source Phase Angle is: -2.4794514576771287 degrees
The Load Phase Angle is: 0 degrees


## Instructions

Do not use any library or module other than those pre-imported.

### Scoring

In each Task,
- a correct use of
  [`scipy.optimize.root()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html)
  merits two (2) points;
- obtaining a correct $V$ merits two (3) points;
- obtaining a correct $\delta_{\text{l}}$ merits two (3) points;
- obtaining a correct $\delta_{\text{s}}$ merits two (3) points;
  and
- obtaining $\boldsymbol{f}\!\left(\boldsymbol{x}\right)$ close enough to $\boldsymbol{0}$ merits six (9) points.


Every violation of an instruction
means a deduction of one (1) point.

All in all, one may earn up to 120 points for this activity.

### Submission

Download this notebook file,
and save with a filename following the pattern
`ACT-09_<Group name>`,
where the group name is as listed in class.
For example, if you belong to the group SixIsEven,
then your notebook should be named `ACT-09_SixIsEven.ipynb`.
Submit your notebook via the classwork platform for this activity in Google Classroom.
Submissions beyond the deadline will not be considered.

Lastly, the use of AI tools to answer this exam is not prohibited,
but it is of ethical interest to disclose such use.
This is in line with the
[MSU Policy on the Fair and Ethical Use of AI and Its Applications](https://www.msumain.edu.ph/wp-content/uploads/2024/05/MSU-Policy-on-Ethical-use-of-AI-Policies.pdf).
As such, please include a brief statement (in a private comment to this classwork)
declaring which and how AI tools are used in your work.

*Last updated by Christian Cahig on 2025-12-11*